This notebook was used to compile all of the available data from the Utah Flux Network stations.  It should only need to be used once, as other notebooks are used to comile the newer data.

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import geopandas as gpd
import urllib
import sys
import pathlib
import glob
import matplotlib.pyplot as plt
from urllib.parse import quote
from sqlalchemy import create_engine
import configparser


import statsmodels.api as sm
#import pingouin as pg
import plotly.express as px





In [2]:
sys.path.append("//")
#sys.path.append("../../Micromet")
import micromet
from micromet import AmerifluxDataProcessor
%matplotlib inline

In [4]:
site_folders = {'US-UTD':'Dugout_Ranch',
                'US-UTB':'BSF',
                'US-UTJ':'Bluff',
                'US-UTW':'Wellington',
                'US-UTE':'Escalante',
                'US-UTM':'Matheson',
                'US-UTP':'Phrag',
                'US-CdM':'Cedar_mesa',
                'US-UTV':'Desert_View_Myton',
                'US-UTN':'Juab'
                }


compdf = {}

am = micromet.AmerifluxDataProcessor()

for key, value in site_folders.items():

    print(key)
    raw_fold = pathlib.Path('H:/UGS_Flux/Data_Downloads/')
    raw_data = am.raw_file_compile(raw_fold, value, search_str = "*Flux_AmeriFluxFormat*.dat")
    if raw_data is not None:
        am_data = micromet.Reformatter(raw_data, data_path="C:/Users/paulinkenbrandt/Documents/GitHub/MicroMet/data/extreme_values.csv")
        am_df = am_data.et_data
        compdf[key] = am_df

        am_df.to_csv(f"../../station_data/{key}_HH_{am_df['TIMESTAMP_START'].values[0]:}_{am_df['TIMESTAMP_END'].values[-1]:}.csv")

        
cdf = pd.concat(compdf,axis=0)
cdf.index.set_names(['stationid','datetime_start'],inplace=True)
#cdf.rename(columns={'level_0':'stationid'},inplace=True)
#cdf.to_parquet('../station_data/all_data.parquet')
for col in cdf.columns:
    cdf.rename(columns={col:col.lower()},inplace=True)


US-UTD
US-UTB
US-UTJ
US-UTW
US-UTE
US-UTM
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Matheson\8442_Flux_AmeriFluxFormat_0.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Matheson\d20211228\8442_Flux_AmeriFluxFormat_0.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Matheson\d20220215\8442_Flux_AmeriFluxFormat_0.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Matheson\d20220323\8442_Flux_AmeriFluxFormat_0.dat
US-UTP


C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:945: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(arr)
C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:945: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(arr)
C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:

US-CdM
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Cedar_mesa\e20241008\45027_Flux_AmeriFluxFormat_6.dat
US-UTV
US-UTN
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Juab\20210622_farm\8441_Flux_AmeriFluxFormat_4.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Juab\combined_farm\8441_Flux_AmeriFluxFormat_4.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Juab\20210826_farmNephi\20210826\8441_Flux_AmeriFluxFormat_11.dat


C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:945: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(arr)
C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:945: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(arr)
C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:

In [3]:
config = configparser.ConfigParser()

config.read('../../secrets/config.ini')

from sqlalchemy import create_engine
import urllib.parse
host = config['DEFAULT']['ip']
pw = config['DEFAULT']['pw']
user = config['DEFAULT']['login']

encoded_password = urllib.parse.quote_plus(pw)

def postconn_et(encoded_password, host='localhost',user='postgres',port='5432',db='groundwater', schema = 'groundwater'):
    connection_text = "postgresql+psycopg2://{:}:{:}@{:}:{:}/{:}?gssencmode=disable".format(user,encoded_password,host,port,db)
    return create_engine(connection_text, connect_args={'options': '-csearch_path={}'.format(schema)})


engine = postconn_et(encoded_password, host=host, user=user)

In [5]:
cdf.to_sql(name = 'amfluxeddy',
           schema='groundwater',
           con=engine,
           if_exists='replace',
           chunksize=2000)

15788